In [141]:
import pandas as pd
df = pd.read_csv("法令名等一覧.csv", encoding="ANSI")
df.head()

,ＩＤ,法令名,条項,法令名.1,法令番号,法令番号_漢数字,法令名_eGov,法令番号_eGov,条番号_漢数字,条番号_数字,条番号_eGov,条見出し,条見出し_eGov
0,1,警備業法（昭和47年法律第117号）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,建築基準法施行令（昭和25年政令第338号）,第１条（用語の定義）第３号,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,災害救助法（昭和22年法律第118号）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,災害対策基本法（昭和36年法律第223号）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,自動車損害賠償保障法（昭和30年法律第97号）,第２条（定義）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
#dfに列を追加する。要素カラだが、文字列とする。
df['項'] = pd.Series(dtype='string')
df['号'] = pd.Series(dtype='string')
df['本則附則の別'] = pd.Series(dtype='string')


#法令名という列の見出しを法令名および法令番号に変更する。
df.rename(columns={'法令名': '法令名および法令番号'}, inplace=True)
#法令名.1という列の見出しを法令名に変更する。
df.rename(columns={'法令名.1': '法令名'}, inplace=True)


In [143]:
#法令名および法令番号の要素を設定する。
df['法令名'] = df['法令名および法令番号'].str.split('（').str[0]
df['法令番号'] = df['法令名および法令番号'].str.split('（').str[1].str.replace('）', '')

#本則附則の別は、条項列の冒頭に「附則」が含まれる場合は「附則」、それ以外の場合は「本則」とする。
df['本則附則の別'] = df['条項'].str.startswith('附則').replace({True: '附則', False: '本則'})

#条番号の列を設定する。条番号は、条項列の「附則第」または「第」から「条」までの文字列を抽出する。
df['条番号_漢数字'] = df['条項'].str.extract(r'(第[0-9０-９]+条(?:の[０-９]+)?)')[0]

#条番号_数字の列を設定する。条番号_数字は、条番号から「の」を「_」に変換し、「第」と「条」を除き、全角を半角に変換した文字列を抽出する。
df['条番号_数字'] = df['条番号_漢数字'].str.replace('第', '').str.replace('条', '').str.replace('の', '_')
#「条番号_数字」の列の要素に、str.maketrans('０１２３４５６７８９', '0123456789')を適用して全角数字を半角数字に変換する
df['条番号_数字'] = df['条番号_数字'].str.translate(str.maketrans('０１２３４５６７８９', '0123456789'))


#条番号_数字の列を設定し終わったので、条番号の列の要素に、convert_arabic_to_egovlawno.pyで定義されたarabic_to_kanji関数を適用して、数字を漢数字に変換する。
from convert_arabic_to_egovlawno import convert_arabic_to_kanji_in_string
df['条番号_漢数字'] = df['条番号_漢数字'].apply(lambda x: convert_arabic_to_kanji_in_string(str(x)) if pd.notnull(x) else x)

#項列と号列にそれぞれの数値を半角で設定する
df['項'] = df['条項'].astype(str).str.extract(r'(第[0-9０-９]+項)')[0].str.replace('第', '').str.replace('項', '').str.translate(str.maketrans('０１２３４５６７８９', '0123456789'))
df['号'] = df['条項'].astype(str).str.extract(r'(第[0-9０-９]+号)')[0].str.replace('第', '').str.replace('号', '').str.translate(str.maketrans('０１２３４５６７８９', '0123456789'))

#法令番号_漢数字列に、法令番号を漢数字に変換した値を設定する。
df['法令番号_漢数字'] = df['法令番号'].apply(lambda x: convert_arabic_to_kanji_in_string(str(x)) if pd.notnull(x) else x)

#条見出し列に、条項の全角かっこで囲まれた部分を抽出する。
df['条見出し'] = df['条項'].str.extract(r'(（.*?）)')[0]

display(df)


,ＩＤ,法令名および法令番号,条項,法令名,法令番号,法令番号_漢数字,法令名_eGov,法令番号_eGov,条番号_漢数字,条番号_数字,条番号_eGov,条見出し,条見出し_eGov,項,号,本則附則の別
0,1,警備業法（昭和47年法律第117号）,NaN,警備業法,昭和47年法律第117号,昭和四十七年法律第百十七号,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,建築基準法施行令（昭和25年政令第338号）,第１条（用語の定義）第３号,建築基準法施行令,昭和25年政令第338号,昭和二十五年政令第三百三十八号,NaN,NaN,第一条,1,NaN,（用語の定義）,NaN,NaN,3,本則
2,3,災害救助法（昭和22年法律第118号）,NaN,災害救助法,昭和22年法律第118号,昭和二十二年法律第百十八号,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,災害対策基本法（昭和36年法律第223号）,NaN,災害対策基本法,昭和36年法律第223号,昭和三十六年法律第二百二十三号,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,自動車損害賠償保障法（昭和30年法律第97号）,第２条（定義）,自動車損害賠償保障法,昭和30年法律第97号,昭和三十年法律第九十七号,NaN,NaN,第二条,2,NaN,（定義）,NaN,NaN,NaN,本則
5,6,自動車損害賠償保障法（昭和30年法律第97号）,第２条第３項,自動車損害賠償保障法,昭和30年法律第97号,昭和三十年法律第九十七号,NaN,NaN,第二条,2,NaN,NaN,NaN,3,NaN,本則
6,7,自動車損害賠償保障法（昭和30年法律第97号）,第２条第４項,自動車損害賠償保障法,昭和30年法律第97号,昭和三十年法律第九十七号,NaN,NaN,第二条,2,NaN,NaN,NaN,4,NaN,本則
7,8,自動車損害賠償保障法（昭和30年法律第97号）,第３条（自動車損害賠償責任）,自動車損害賠償保障法,昭和30年法律第97号,昭和三十年法律第九十七号,NaN,NaN,第三条,3,NaN,（自動車損害賠償責任）,NaN,NaN,NaN,本則
8,9,消防法（昭和23年法律第186号）,NaN,消防法,昭和23年法律第186号,昭和二十三年法律第百八十六号,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,臓器の移植に関する法律（平成９年法律第104号）,第６条（臓器の摘出）,臓器の移植に関する法律,平成９年法律第104号,平成九年法律第百四号,NaN,NaN,第六条,6,NaN,（臓器の摘出）,NaN,NaN,NaN,本則


In [144]:
#法令名_eGovと法令番号_eGovの列を設定
#一度読み込んだretrieve_law_no_or_name.pyをリロード
import importlib
importlib.reload(importlib.import_module('retrieve_law_no_or_name'))
from retrieve_law_no_or_name import RetrieveLaw
retriever = RetrieveLaw()
df['法令名_eGov'] = df['法令番号_漢数字'].apply(lambda x: retriever.get_law_name_or_no("LawNo",str(x)) if pd.notnull(x) else x)
df['法令番号_eGov'] = df['法令名'].apply(lambda x: retriever.get_law_name_or_no("LawName",str(x)) if pd.notnull(x) else x)

In [145]:
#条見出しの列の要素に対応する条番号を、条番号_eGov列に設定する。そのためには、RetrieveArticle()を、法令番号_漢数字列の要素を引数にインスタンス化し、RetrieveArticleNoOrCaption().get_article_no()に条見出しを引数にして適用する。
importlib.reload(importlib.import_module('retrieve_article_no_or_caption'))
from retrieve_article_no_or_caption import RetrieveArticle as Ret
from retrieve_article_no_or_caption import RetrieveSupplyArticle as RetSup

def exists_article_caption_and_get_no(law_no, article_caption):
    try:
        return Ret(law_no).get_article_no(article_caption)
    except Exception:
        return None

def exists_article_no_and_get_caption(law_no, article_no):
    try:
        return Ret(law_no).get_article_caption(article_no)
    except Exception:
        return None
    
def exists_article_caption_and_get_no_spp(law_no, article_caption):
    try:
        retsup = RetSup(law_no)
        retsup.amend_data()
        return retsup.get_article_no(article_caption)
    except Exception:
        return None

def exists_article_no_and_get_caption_spp(law_no, article_no):
    try:    
        retsup = RetSup(law_no)
        retsup.amend_data()
        # print(retsup.data)
        # print(law_no)
        # print(article_no)
        # print(retsup.get_article_caption(article_no))
        return retsup.get_article_caption(article_no)
    except Exception:
        return None
    
#df['本則附則の別']が「附則」の場合は、RetrieveSupplyArticleを使用し、「本則」の場合はRetrieveArticleを使用する。
df['条番号_eGov'] = df.apply(lambda row: exists_article_caption_and_get_no_spp(row['法令番号_漢数字'], row['条見出し']) if row['本則附則の別'] == '附則' else exists_article_caption_and_get_no(row['法令番号_漢数字'], row['条見出し']) , axis=1)
df['条見出し_eGov'] = df.apply(lambda row: exists_article_no_and_get_caption_spp(row['法令番号_漢数字'], row['条番号_漢数字']) if row['本則附則の別'] == '附則' else exists_article_no_and_get_caption(row['法令番号_漢数字'], row['条番号_漢数字']) , axis=1)



https://laws.e-gov.go.jp/api/1/lawdata/昭和四十七年法律第百十七号
https://laws.e-gov.go.jp/api/1/lawdata/昭和二十五年政令第三百三十八号
https://laws.e-gov.go.jp/api/1/lawdata/昭和二十二年法律第百十八号
https://laws.e-gov.go.jp/api/1/lawdata/昭和三十六年法律第二百二十三号
https://laws.e-gov.go.jp/api/1/lawdata/昭和三十年法律第九十七号
https://laws.e-gov.go.jp/api/1/lawdata/昭和三十年法律第九十七号
https://laws.e-gov.go.jp/api/1/lawdata/昭和三十年法律第九十七号
https://laws.e-gov.go.jp/api/1/lawdata/昭和三十年法律第九十七号
https://laws.e-gov.go.jp/api/1/lawdata/昭和二十三年法律第百八十六号
https://laws.e-gov.go.jp/api/1/lawdata/平成九年法律第百四号
https://laws.e-gov.go.jp/api/1/lawdata/平成九年法律第百四号
https://laws.e-gov.go.jp/api/1/lawdata/平成九年法律第百四号
https://laws.e-gov.go.jp/api/1/lawdata/昭和五十三年法律第七十三号
https://laws.e-gov.go.jp/api/1/lawdata/昭和四十一年法律第七十三号
https://laws.e-gov.go.jp/api/1/lawdata/昭和二十六年運輸省令第六十七号
https://laws.e-gov.go.jp/api/1/lawdata/平成十四年国土交通省告示第六百十九号
Error: 404
https://laws.e-gov.go.jp/api/1/lawdata/平成十四年国土交通省告示第六百十九号
Error: 404
https://laws.e-gov.go.jp/api/1/lawdata/昭和三十五年法律第百五号
https://laws.e-gov.go

In [ ]:
#「法令名」列と「法令名_eGov」列の名前が同じ、かつ、「法令番号_漢数字」列と「法令番号_eGov」列の名前が同じ場合は、「法令名の整合」列にTrue、そうでなければFalseを設定する。
df['法令名の整合'] = (df['法令名'] == df['法令名_eGov']) & (df['法令番号_漢数字'] == df['法令番号_eGov'])

#「条番号_漢字」列と「条番号_数字_eGov」列の名前が同じ、かつ、「条見出し」列と「条見出し_eGov」列の名前が同じ場合は、「条見出しの整合」列にTrue、そうでなければFalseを設定する。なお、「条項」がNaNの場合には、Trueとする
df['条見出しの整合'] = (df['条見出し'] == df['条見出し_eGov']) & (df['条番号_漢数字'] == df['条番号_eGov']) | df['条項'].isna()

from check_paragraph_or_item import check_item_or_paragraph
importlib.reload(importlib.import_module('check_paragraph_or_item'))
#条項チェックという列を追加、値はbool型で一律NaN
df['条項チェック'] = pd.Series(dtype='float')

#条項チェック列に、法令番号、条番号、項番号、号番号を引数にしてcheck_item_or_paragraph()を適用する。
df['条項チェック'] = df.apply(
    lambda row: check_item_or_paragraph(
        row['法令番号_漢数字'],
        row['条番号_数字'],
        row['項'],
        row['号']
    ),
    axis=1
)
display(df)


,ＩＤ,法令名および法令番号,条項,法令名,法令番号,法令番号_漢数字,法令名_eGov,法令番号_eGov,法令名の整合,条番号_漢数字,条番号_数字,条番号_eGov,条見出し,条見出し_eGov,条見出しの整合,項,号,本則附則の別,条項チェック
0,10,臓器の移植に関する法律（平成９年法律第104号）,第６条（臓器の摘出）,臓器の移植に関する法律,平成９年法律第104号,平成九年法律第百四号,臓器の移植に関する法律,平成九年法律第百四号,True,第六条,6,第六条,（臓器の摘出）,（臓器の摘出）,True,NaN,NaN,本則,None
1,11,臓器の移植に関する法律（平成９年法律第104号）,第６条第４項,臓器の移植に関する法律,平成９年法律第104号,平成九年法律第百四号,臓器の移植に関する法律,平成九年法律第百四号,True,第六条,6,None,NaN,（臓器の摘出）,False,4,NaN,本則,True
2,12,臓器の移植に関する法律（平成９年法律第104号）,附則第11条,臓器の移植に関する法律,平成９年法律第104号,平成九年法律第百四号,臓器の移植に関する法律,平成九年法律第百四号,True,第十一条,11,None,NaN,None,False,NaN,NaN,附則,None
